In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np

In [41]:
#filename = 'normalTrafficTest.txt'
#filename = 'anomalousTrafficTest.txt' 
#filename = 'normalTrafficTraining.txt'

In [42]:
data = pd.read_csv('../data/'+filename,sep='\n\n',header=None,names=['sentence'],)

print(data)

C:\Users\debon\AppData\Local\Temp\ipykernel_8260\3741842360.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('../data/'+filename,sep='\n\n',header=None,names=['sentence'],)


                                                 sentence
0       GET http://localhost:8080/tienda1/index.jsp HT...
1       User-Agent: Mozilla/5.0 (compatible; Konqueror...
2                                        Pragma: no-cache
3                                 Cache-control: no-cache
4       Accept: text/xml,application/xml,application/x...
...                                                   ...
419995        Accept-Charset: utf-8, utf-8;q=0.5, *;q=0.5
419996                                Accept-Language: en
419997                               Host: localhost:8080
419998  Cookie: JSESSIONID=A70DD1BA160B294CB5E1C2D8FAE...
419999                                  Connection: close

[420000 rows x 1 columns]


In [43]:
data.value_counts().head(20)

sentence                                                                                                   
Accept-Charset: utf-8, utf-8;q=0.5, *;q=0.5                                                                    36000
User-Agent: Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko)                            36000
Accept-Language: en                                                                                            36000
Accept: text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5    36000
Cache-control: no-cache                                                                                        36000
Connection: close                                                                                              36000
Host: localhost:8080                                                                                           36000
Accept-Encoding: x-gzip, x-deflate, gzip, deflate                        

In [44]:
top_10_freq_sent = [x[0] for x in data.value_counts().head(10).index.tolist()]
print([x[:x.index(':')] for x in top_10_freq_sent])

['Accept-Charset', 'User-Agent', 'Accept-Language', 'Accept', 'Cache-control', 'Connection', 'Host', 'Accept-Encoding', 'Pragma', 'Content-Type']


In [45]:
sent_to_remv = ['POST http','PUT http','Cookie: ','Content-Length: ','Accept-Charset', 'User-Agent', 'Accept-Language', 'Accept', 'Cache-control', 'Connection', 'Host', 'Accept-Encoding', 'Pragma', 'Content-Type']

for freq_item in sent_to_remv:
    data = data.drop(data[data['sentence'].str.startswith(freq_item)].index)

In [46]:
print(data.value_counts().head(40))
print(data.describe())

sentence                                                                                                                                 
B2=Vaciar+carrito                                                                                                                            1000
errorMsg=Credenciales+incorrectas                                                                                                            1000
GET http://localhost:8080/tienda1/miembros/imagenes/ogono.jpg HTTP/1.1                                                                       1000
GET http://localhost:8080/tienda1/miembros/imagenes/zarauz.jpg HTTP/1.1                                                                      1000
GET http://localhost:8080/tienda1/miembros/index.jsp HTTP/1.1                                                                                1000
GET http://localhost:8080/tienda1/miembros/salir.jsp HTTP/1.1                                                                       

In [47]:
from urllib.parse import unquote
import re

#REPLACE_BY_SPACE_RE = re.compile('[^0-9a-zA-Z _/(){}\[\]\|@,;]')
#REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
REPLACE_BY_SPACE_RE = re.compile('[&=+]')

def extract_text(text):
    if text.startswith('GET http'):
        url = text.split()[1]
        if '?' in url:
            text = url[url.index('?')+1:].strip()
        else:
            text = ''
    text = unquote(text) 
    
    text = text.lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    return text

data['sentence'] =data['sentence'].apply(extract_text)

In [48]:
data.describe()

,sentence
count,36000
unique,4802
top,
freq,20000


In [49]:
data = data.drop(data[data['sentence'].str.strip(' \t\n\r') ==''].index)

In [50]:
data=data.reset_index().drop('index',axis=1)

In [51]:
data.describe()

,sentence
count,16000
unique,4801
top,errormsg credenciales incorrectas
freq,2000


In [52]:
data.to_csv('../data/processed/'+filename,index=None,header=None,)